In [32]:
import os
import pandas as pd
from Data_loader import Dataset_Custom
import argparse
import warnings
from tools import EarlyStopping
from torch.utils.data import Dataset, DataLoader
from utils import get_data
from Model import ANN
from torch import nn
import torch
import copy
from tqdm import tqdm
import random 
import numpy as np
import torch.nn.functional as F
warnings.filterwarnings("ignore")

In [33]:
parser_train = argparse.ArgumentParser(description='FL')
parser_train.add_argument('--root_path', type=str, default='../Data/GFC12/')
parser_train.add_argument('--dataset_paths', type=list, default=["wf1", "wf2", "wf3", "wf4", "wf5", "wf6", "wf7"])
parser_train.add_argument('--number_clients', type=int, default=7)
parser_train.add_argument('--seq_len', type=int, default=24*4)
parser_train.add_argument('--pred_len', type=int, default=12)
parser_train.add_argument('--label_len', type=int, default=0)
parser_train.add_argument('--train_length', type=int, default=16800)
parser_train.add_argument('--target', type=str, default='target')
parser_train.add_argument('--scale', type=bool, default=True)
parser_train.add_argument('--inverse', type=bool, default=True)
parser_train.add_argument('--k', type=int, default=2)

parser_train.add_argument('--lr', type=float, default=1e-4)
parser_train.add_argument('--fine_tune_lr', type=float, default=1e-5)
parser_train.add_argument('--global_epochs', type=int, default=200)
parser_train.add_argument('--local_epochs', type=int, default=1)
parser_train.add_argument('--fine_tune_epochs', type=int, default=20)
parser_train.add_argument('--patience', type=int, default=3)
parser_train.add_argument('--fed_patience', type=int, default=3)
parser_train.add_argument('--hidden_layers', type=list, default=[64,64,64])
parser_train.add_argument('--input_size', type=int, default=293)
parser_train.add_argument('--output_size', type=int, default=9)


parser_train.add_argument('--warm_up_epochs', type=int, default=15)
parser_train.add_argument('--selection_epochs', type=int, default=4)
parser_train.add_argument('--importance', type=float, default=0)
parser_train.add_argument('--batch_size', type=int, default=64)
parser_train.add_argument('--device', type=str, default='cuda:1' if torch.cuda.is_available() else 'cpu')
parser_train.add_argument('--forecasting_mode', type=str, default='prob')
parser_train.add_argument('--model_type', type=str, default='NN')
parser_train.add_argument('--model_save_path', type=str, default='../Model2/', help='location of model checkpoints')
parser_train.add_argument('--quantiles', type=list, default=[0.1,0.2,0.3,0.4, 0.5,0.6,0.7,0.8, 0.9])
parser_train.add_argument('--decay', type=float, default=0.75)
args_train = parser_train.parse_args(args=[])

In [83]:
from Server import  Server_mul,Server
from Clients import Client_mul,Client
from Train import Trainer,EWCLoss,evaluation,PinballLoss
from utils import get_data, plot_prob_result,seed_everything

ImportError: cannot import name 'Winkler_score' from 'Train' (/home/zyz/IET 联邦概率/Code 12h/Train.py)

In [85]:

class Winkler_score():
    def __init__(self, alpha: float, reduction: str = 'none'):
        self.reduction = reduction
        self.alpha = alpha

    def score(self,l_interval,u_interval,y_t):
        score = u_interval - l_interval
        if y_t < l_interval:
            score += ((2/self.alpha) * (l_interval - y_t))
        elif y_t > u_interval:
            score += ((2/self.alpha) * (y_t - u_interval))
        return score

    def __call__(self, output: torch.Tensor, target: torch.Tensor):
        if self.alpha==0.2:
            u_interval_list = output[-1]
            l_interval_list = output[0]
        elif self.alpha==0.4:
            u_interval_list = output[-2]
            l_interval_list = output[1]
        elif self.alpha==0.6:
            u_interval_list = output[-3]
            l_interval_list = output[2]

        loss = torch.Tensor([
            self.score(l_interval, u_interval, y_t) 
            for l_interval, u_interval, y_t in zip(l_interval_list, u_interval_list, target)
        ])
        if self.reduction == 'sum':
            loss = loss.sum()
        if self.reduction == 'mean':
            loss = loss.mean()

        return loss

In [35]:
seed_everything(0)

In [36]:
clients=[]
for path in tqdm(args_train.dataset_paths):
    args_temp=copy.deepcopy(args_train)
    args_temp=copy.deepcopy(args_train)
    args_temp.dataset_paths=path
    clients.append(Client_mul(args_temp))

100%|██████████| 7/7 [00:04<00:00,  1.46it/s]


In [37]:
server = Server_mul(args_train,clients)

In [38]:
server.fed_train() 

test performance: [0.1464793815417853, 0.14697975615575298, 0.1750156776851987, 0.15993108314602342, 0.1522533442978173, 0.16320744794729639, 0.1622745959113722]
False [False, False]
ewc: False
wf1 model: 0
Epoch: 0 | Loss: 0.0799
wf1 model: 1
Epoch: 0 | Loss: 0.0657
ewc: False
wf2 model: 0
Epoch: 0 | Loss: 0.0697
wf2 model: 1
Epoch: 0 | Loss: 0.0876
ewc: False
wf3 model: 0
Epoch: 0 | Loss: 0.0923
wf3 model: 1
Epoch: 0 | Loss: 0.1041
ewc: False
wf4 model: 0
Epoch: 0 | Loss: 0.0900
wf4 model: 1
Epoch: 0 | Loss: 0.0904
ewc: False
wf5 model: 0
Epoch: 0 | Loss: 0.0869
wf5 model: 1
Epoch: 0 | Loss: 0.0889
ewc: False
wf6 model: 0
Epoch: 0 | Loss: 0.0953
wf6 model: 1
Epoch: 0 | Loss: 0.0700
ewc: False
wf7 model: 0
Epoch: 0 | Loss: 0.0815
wf7 model: 1
Epoch: 0 | Loss: 0.0922
Client 0 test loss: [0.07542034006624851, 0.07580236148721767]
Client 1 test loss: [0.07907687141647879, 0.07940910672241787]
Client 2 test loss: [0.0974197729297404, 0.09800436446126902]
Client 3 test loss: [0.08570393217

[ANN(
   (hidden_layers): ModuleList(
     (0): Linear(in_features=293, out_features=64, bias=True)
     (1-2): 2 x Linear(in_features=64, out_features=64, bias=True)
   )
   (relu): ReLU()
   (output): Linear(in_features=64, out_features=9, bias=True)
 ),
 ANN(
   (hidden_layers): ModuleList(
     (0): Linear(in_features=293, out_features=64, bias=True)
     (1-2): 2 x Linear(in_features=64, out_features=64, bias=True)
   )
   (relu): ReLU()
   (output): Linear(in_features=64, out_features=9, bias=True)
 )]

In [39]:
#0.5
fed_local_losses=[]
fed_local_preds=[]
fed_local_models=[]
for i in range(args_train.number_clients):
    fed_local_pred,fed_local_loss,fed_local_model=clients[i].fed_local_evaluation()
    fed_local_losses.append(fed_local_loss[server.index_set[i]])
    fed_local_preds.append(fed_local_pred[server.index_set[i]])
    fed_local_models.append(fed_local_model[server.index_set[i]])
print(fed_local_losses)
print(np.mean(fed_local_losses))

[0.07019277994380627, 0.06390050535205088, 0.07960541857636139, 0.07376481334946744, 0.07278922048384605, 0.06719664158299565, 0.08431435201623261]
0.07310910447210861


In [45]:
import pickle

# Save the server object
with open('../result/12/server.pkl', 'wb') as f:
    pickle.dump(server, f)

# Save the clients object
with open('../result/12/clients.pkl', 'wb') as f:
    pickle.dump(clients, f)